In [1]:
# Use this code if using Colab to run Selenium

# Make sure to go to Runtime -> Change runtime and set GPU as hardware accelerator

# !kill -9 -1 # Use this line to delete this VM and start a new one.
# The above line deletes all files and folders from the current VM and allocates a new one.

#Selenium is an open-source tool that automates web browsers.
!pip install selenium
!apt-get -q update   #Used to handle installation and removal of softwares and libraries
!apt install -yq chromium-chromedriver #ChromeDriver is a separate executable that Selenium WebDriver uses to control Chrome.
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 28.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00
zsh:1: command not found: apt-get
The operation couldn’t be completed. Unable to locate a Java Runtime that supports apt.
Please visit http://www.java.com for information on installing Java.

cp: /usr/lib/chromium-browser/chromedriver: No such file or directory


**Part A**

In [1]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.by import By
#WebDriver is a browser automation framework that works with open source APIs.
#The framework operates by accepting commands, sending those commands to a browser, and interacting with applications.
chrome_options = webdriver.ChromeOptions()
#headless means running chrome with chrome.exe
chrome_options.add_argument('--headless')

chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [2]:
wd = webdriver.Chrome(options=chrome_options)
wd.get('https://www.beeradvocate.com/beer/popular/')
link_list=[]
for i in range(2,252):
  xpath = f"//*[@id='ba-content']/table/tbody/tr[{i}]/td[2]/a"
  try:
        hrefs=wd.find_element(By.XPATH, xpath)
        href = hrefs.get_attribute("href")
        link_list.append(href)
  except Exception as e:
        print(f"Error retrieving link {i}: {str(e)}")


In [3]:
link_list

['https://www.beeradvocate.com/beer/profile/1199/11757/',
 'https://www.beeradvocate.com/beer/profile/64/2093/',
 'https://www.beeradvocate.com/beer/profile/287/1093/',
 'https://www.beeradvocate.com/beer/profile/863/7971/',
 'https://www.beeradvocate.com/beer/profile/46317/16814/',
 'https://www.beeradvocate.com/beer/profile/199/29619/',
 'https://www.beeradvocate.com/beer/profile/1199/19960/',
 'https://www.beeradvocate.com/beer/profile/287/17112/',
 'https://www.beeradvocate.com/beer/profile/1146/10672/',
 'https://www.beeradvocate.com/beer/profile/112/412/',
 'https://www.beeradvocate.com/beer/profile/140/276/',
 'https://www.beeradvocate.com/beer/profile/64/6108/',
 'https://www.beeradvocate.com/beer/profile/147/88/',
 'https://www.beeradvocate.com/beer/profile/140/30420/',
 'https://www.beeradvocate.com/beer/profile/26/64228/',
 'https://www.beeradvocate.com/beer/profile/147/84596/',
 'https://www.beeradvocate.com/beer/profile/43153/92/',
 'https://www.beeradvocate.com/beer/profi

In [4]:
#importing the libraries needed
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import randint
import re
from selenium.common.exceptions import NoSuchElementException

#Declaring the headers
headers = {"Accept-Language": "en-US,en;q=0.5"}

#declaring the list of empty variables, So that we can append the data overall

beer = []
rating2 = []
rating = []
review=[]


# Loop through the list of URLs
for url in link_list:
    try:
        # Send an HTTP GET request for each URL
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')
        beer_data = soup.findAll('div', attrs = {'id': 'rating_fullview_content_2'})
        title_data = soup.find('title').text
            #Will send request in randon 2-8 sec to the imbd website
            #sleep(randint(2,8))
        for store in beer_data:
            parts = title_data.split('|')
            if len(parts) > 0:
                beer_name = parts[0].strip()
            beer.append(beer_name)
            
            try:
                rate2 = store.find('span', class_ = "BAscore_norm")
                if rate2:
                    rate2 = rate2.text
                    #rate2 = [float(rte) for rte in rate2 if float(rte) <= 5 and float(rte) > 0]
                    #rate2_result = float(sum(rate2))
                    rate2_result = float(''.join(map(str, rate2)))
                    rating2.append(rate2_result)
                else:
                    rating2.append(None)
            except NoSuchElementException:
                break

            rate = store.find('span', class_ = "muted").text
            rate = re.findall(r"[-+]?(?:\d*\.*\d+)", rate)
            rate = [float(rte) for rte in rate if float(rte) <= 5 and float(rte) > 0]
            rate_result = float(sum(rate))
            rating.append(rate_result)

            try:
                describe = store.find('div')
                if describe:
                    describe = describe.text
                    review.append(describe)
                else:
                    review.append(None)
            except NoSuchElementException:
                break

    except Exception as e:
        # Handle any exceptions that may occur during the request
        print(f"An error occurred while fetching data from {url}: {str(e)}")
    

#creating a dataframe
beer_list = pd.DataFrame({"Beer": beer, "Review_Rating": rating2, "Rating" : rating, "Review": review})

def combine_floats(row):
    if isinstance(row['Review_Rating'], float) and row['Review_Rating']>0:
        return row['Review_Rating']
    elif isinstance(row['Rating'], float)and row['Rating']>0:
        return row['Rating']
    else:
        return None  # Handle the case where neither column has a float

beer_list['Combined_Rating'] = beer_list.apply(combine_floats, axis=1)
beer_df = beer_list[['Beer','Combined_Rating','Review']]


In [5]:
beer_df

,Beer,Combined_Rating,Review
0,Breakfast Stout,4.40,None
1,Breakfast Stout,4.25,None
2,Breakfast Stout,4.94,Amazing taste for a dark stout beer!
3,Breakfast Stout,5.00,"Well, well, well, well, well. Stout lovers of ..."
4,Breakfast Stout,4.20,"Had on tap at Bulldog Saloon Lynnwood, WA."
...,...,...,...
4995,Troegenator,4.00,None
4996,Troegenator,4.18,Awesome beer for the winter season although I ...
4997,Troegenator,4.00,None
4998,Troegenator,4.25,"""Freshest by 3/29/23""; drank 12/23/22 @ Pokemo..."


In [188]:
beer_df.to_csv('BeerAdvocateRandR.csv')

Part B

In [14]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize.sonority_sequencing import punctuation
from nltk.tokenize import RegexpTokenizer, word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
data = beer_df

# Convert 'Message' column to lowercase
data['Review'] = data['Review'].astype(str)
data['Review'] = data['Review'].apply(lambda x: x.lower())

# Get a list of English stopwords from NLTK
stop = stopwords.words('english')

# Tokenize each 'MappedMessage' and remove punctuation
data['tokenized_message'] = data['Review'].apply(lambda s: word_tokenize(re.sub(r'[^\w\s]', ' ', s)))

# Display the updated DataFrame 'data'
data

,Beer,Combined_Rating,Review,tokenized_message,tokenized_message_unique
0,Breakfast Stout,4.40,none,[none],{none}
1,Breakfast Stout,4.25,none,[none],{none}
2,Breakfast Stout,4.94,amazing taste for a dark stout beer!,"[amazing, taste, for, a, dark, stout, beer]","{dark, beer, amazing, stout, taste}"
3,Breakfast Stout,5.00,"well, well, well, well, well. stout lovers of ...","[well, well, well, well, well, stout, lovers, ...","{stout, unite, take, lovers, well, world}"
4,Breakfast Stout,4.20,"had on tap at bulldog saloon lynnwood, wa.","[had, on, tap, at, bulldog, saloon, lynnwood, wa]","{wa, lynnwood, saloon, tap, bulldog}"
...,...,...,...,...,...
4995,Troegenator,4.00,none,[none],{none}
4996,Troegenator,4.18,awesome beer for the winter season although i ...,"[awesome, beer, for, the, winter, season, alth...","{amateurs, two, season, summer, ipa, easily, w..."
4997,Troegenator,4.00,none,[none],{none}
4998,Troegenator,4.25,"""freshest by 3/29/23""; drank 12/23/22 @ pokemo...","[freshest, by, 3, 29, 23, drank, 12, 23, 22, p...","{drank, nice, thick, head, quite, 22, brown, n..."


In [15]:
# Remove stopwords from each tokenized message
data['tokenized_message'] = data['tokenized_message'].apply(lambda words: [x for x in words if x not in stop])

# Create a list of unique words in each tokenized message
data['tokenized_message_unique'] = data['tokenized_message'].apply(lambda s: (set(s)))

# Display the updated DataFrame 'data'
data

,Beer,Combined_Rating,Review,tokenized_message,tokenized_message_unique
0,Breakfast Stout,4.40,none,[none],{none}
1,Breakfast Stout,4.25,none,[none],{none}
2,Breakfast Stout,4.94,amazing taste for a dark stout beer!,"[amazing, taste, dark, stout, beer]","{dark, beer, amazing, stout, taste}"
3,Breakfast Stout,5.00,"well, well, well, well, well. stout lovers of ...","[well, well, well, well, well, stout, lovers, ...","{stout, unite, take, lovers, well, world}"
4,Breakfast Stout,4.20,"had on tap at bulldog saloon lynnwood, wa.","[tap, bulldog, saloon, lynnwood, wa]","{wa, lynnwood, saloon, tap, bulldog}"
...,...,...,...,...,...
4995,Troegenator,4.00,none,[none],{none}
4996,Troegenator,4.18,awesome beer for the winter season although i ...,"[awesome, beer, winter, season, although, easi...","{amateurs, two, season, summer, ipa, easily, w..."
4997,Troegenator,4.00,none,[none],{none}
4998,Troegenator,4.25,"""freshest by 3/29/23""; drank 12/23/22 @ pokemo...","[freshest, 3, 29, 23, drank, 12, 23, 22, pokem...","{drank, nice, thick, head, quite, 22, brown, n..."


In [11]:
# Calculate the frequency of unique words
all_items = []
for s in data['tokenized_message_unique']:
    all_items.extend(s)

# Now you can use nltk.FreqDist or any other counting method on all_items list
# Create a frequency distribution using NLTK's FreqDist
freq_dist = nltk.FreqDist(all_items)
# Remove none since this signified rows without a review
freq_dist.pop('none')
# Display the frequency distribution
freq_dist

FreqDist({'head': 1000, 'beer': 835, 'taste': 814, 'good': 564, 'malt': 510, 'carbonation': 474, 'aroma': 468, 'one': 462, 'pours': 459, 'dark': 458, ...})

In [13]:
# Print the first 150 records
for word, frequency in freq_dist.most_common(25):
    print(f"{word}: {frequency}")

head: 1000
beer: 835
taste: 814
good: 564
malt: 510
carbonation: 474
aroma: 468
one: 462
pours: 459
dark: 458
like: 456
finish: 447
overall: 447
light: 429
sweet: 427
medium: 422
lacing: 419
nice: 410
bottle: 392
white: 391
body: 386
flavor: 373
glass: 362
bitterness: 360
well: 354


Top 3 Attributes:
head, malt, carbonation